# Question 3

In [ ]:
using Plots, DataFrames, CSV, GLM
using Optim, Distributions, Random, ForwardDiff
using LinearAlgebra, StatsFuns, FixedEffectModels

In [ ]:
df = DataFrame(CSV.File("data/ps1_ex3.csv"));

# I need to get the 0 share in each market 
gdf = groupby(df, :market)
gdf = combine(gdf, :Shares => sum)
gdf[!,:shares_0] = 1 .- gdf[!,:Shares_sum];
df = leftjoin(df, gdf, on=:market);
df[!,:log_sj_s0] = log.(df[!,:Shares]./df[!,:shares_0]);

# Part 1: Estimating $(\alpha, \beta)$

$$ ln\left(\frac{s_{jt}}{s_0}\right) = - \alpha p_{jt} + x_{jt}' \beta + \xi_{jt} + \varepsilon_{ijt} $$

In [ ]:
estimates = reg(df, @formula(log_sj_s0 ~ x + (Prices ~ z)))
α = -coef(estimates)[3];
β = coef(estimates)[2];

# Part 2: Finding price elasticities

$$\eta_{jkt} = \frac{\partial s_{jt}}{\partial p_{kt}} \frac{p_{kt}}{s_{jt}} = \begin{cases}
        \alpha p_{jt} (1-s_{jt}) & j=k \\
        - \alpha p_{kt} s_{jt} & j\neq k
    \end{cases}$$

In [ ]:
# reshape dataframe of prices and shares
share_array = Array(unstack(df, :market, :Product, :Shares)[!,2:7]);
price_array = Array(unstack(df, :market, :Product, :Prices)[!,2:7]);

In [ ]:
# for each market, we should get a JxJ matrix 
T = size(share_array)[1]
J = size(share_array)[2]

η = zeros(T, J^2)

for t in 1:T
    iter = 1
    for j in 1:J
        for k in 1:J
            if j == k 
                η[t, iter] = α * price_array[t,j] * (1-share_array[t,k])
            else 
                η[t, iter] = -1 * α * price_array[t,k] * share_array[t,j]
            end
        iter += 1
        end
    end
end

# average across each market
η̄ = reshape(mean(η, dims=1), 6, 6)

# Part 3: Recover marginal costs
Assuming that each product-market is a different firm, we can obtain the following from the FOC from the firm optimization:
$$ s_{jt}(p) + (p_{jt} - mc_{jt}) \frac{\partial s_{jt}}{\partial p_{kt}} = 0 $$
which implies:
$$ s_{jt}(p) \frac{1}{\frac{\partial s_{jt}}{\partial p_{kt}}} + p_{jt} = mc_{jt} $$

In [ ]:
# own partial
df[!,:own_partial] = α .* df[!,:Shares] .* (1 .- df[!,:Shares]);
df[!,:mc] = df[!,:Shares] ./ df[!,:own_partial] .+ df[!,:Prices];

# Part 4: Product 1 exits the market
I think the following works:

- The regression predicts $ln(S_j/S_0)$, where $S_0 = 1 - \sum_k S_k$. Note $\sum_j S_j / S_0 = \frac{1-S_0}{S_0}$:
$$ \exp\left(\ln\left(\frac{S_j}{S_0}\right)\right) = \frac{S_j}{S_0} \\  \frac{\frac{S_j}{S_0}}{ \sum_j \frac{S_j}{S_0}} =  \frac{S_j}{1-S_0} = \frac{S_j}{\sum_k S_k} $$

In [ ]:
df_noprod1 = filter(row -> (row.Product != 1),  df)
df_noprod1[!,:blah] = exp.(predict(estimates, df_noprod1) ./ df_noprod1[!,:shares_0]) ;
df_noprod1[!,[:market, :Product, :blah]]

In [ ]:
df_noprod1 = filter(row -> (row.Product != 1),  df)

iterate = function(tol, model, data, α)
    
    err = 1e10
    iter = 0
    while err > tol 
        
        old_price = data[!,:Prices]
        
        # predict s_jt Q: how do I recover s_j?
        data[!,:new_sjs0] = exp.(predict(model, data))
        gdf = groupby(data, :market)
        gdf = combine(data, :new_sjs0 => sum)
        gdf[!,:shares_0] = 1 .- gdf[!,:new_sjs0];

        # use profit maximization to get new prices
        data[!,:own_partial] = α .* data[!,:Shares] .* (1 .- data[!,:Shares])
        data[!,:Prices] = data[!,:Shares] ./ data[!,:own_partial] .- data[!,:mc] 
        
        err = maximum(abs.(old_price .- data[!,:Prices]))
        iter += 1
    end
    return data, iter, err
end

iterate(1e-10, estimates, df_noprod1, α)

In [ ]:
err = 1e10
df_noprod1 = filter(row -> (row.Product != 1),  df)
data = df_noprod1
tol = 1e-10
iter = 0
    while err > tol 
        
        old_price = data[!,:Prices]
        # predict s_jt Q: how do I recover s_j?
        data[!,:new_sj] = exp.(predict(estimates, data))
        #gdf = groupby(data, :market)
        #gdf = combine(data, :new_sjs0 => sum)
        #gdf[!,:shares_0] = 1 .- gdf[!,:new_sjs0];

        # use profit maximization to get new prices
        data[!,:own_partial] = α .* data[!,:new_sj] .* (1 .- data[!,:new_sj])
        data[!,:Prices] = data[!,:new_sj] ./ data[!,:own_partial] .- data[!,:mc] 
        #println(data[!,:Prices] )

        err = maximum(abs.(old_price .- data[!,:Prices]))
        iter+=1
        println(abs.(old_price .- data[!,:Prices]))
        println(iter)
    end